In [176]:
from xlrd import open_workbook
import urllib
from Bio import Entrez
import xml.etree.ElementTree as ET
import xml.dom.minidom
import pandas as pd

Entrez.email = "kansara3@illinois.edu"

wb = open_workbook('D:/GRA/Data/PMIDs Mapping Results/PMIDs Mapping Results/Unsuccessful Mapping/Conclusion_changed_unsuccessful_mapping.xlsx')
values = []

for s in wb.sheets():
    #print 'Sheet:',s.name
    for row in range(1, s.nrows):
        col_names = s.row(0)
        col_value = []
        for name, col in zip(col_names, range(s.ncols)):
            value  = (s.cell(row,col).value)
            try : value = str(int(value))
            except : pass
            col_value.append((name.value, value))
        values.append(col_value)

# Dctionary for counting number of articles matched on PubMed
match_count = {}
article_count = {}
match_count["counter"] = 0
article_count["counter"] = 0

article_authors_list = []
article_title_list = []
article_volume_list = []
article_pages_list = []
article_journal_list = []
article_year_list = []
article_pub_type_list = []
PMID_list = []
for value in values:
    article_count["counter"] += 1
    
    key = "citation_" + str(count)
    # getting title
    title = value[2][1]
    
    #getting first author
    authors = value[4][1].split(",")
    first_author = authors[0]
    #first_author_string = " AND " + first_author
    first_author = urllib.parse.quote_plus(first_author)
    
    # get year
    year = value[11][1]
    year = urllib.parse.quote_plus(year)
    
    # Encode the article title to pass it as the url compatible text to the PubMed API
    title1 = urllib.parse.quote_plus(title)
        
    # Pass article title to PubMed API to get articles' metadata in XML format
    link = "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=xml&field=title&term=" + title1 + "[title]+AND+" + first_author + "[author]+AND+" + year + "[pdat]"
#     print(link)
    
    f = urllib.request.urlopen(link)
    tree = ET.parse(f)
    doc = tree.getroot()    

    # Find tag with PMID or list of PMIDs

    for child in doc:
        if child.tag == "IdList":
            Id_List = []
            
            
            for step_child in child:
                if step_child.tag == "Id":
                    Id_List.append(step_child.text)
                    print(step_child.text)
                    PMID_list.append(Id_List)
                    
                    link1 = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=" + step_child.text + "&version=2.0"
                    print(link1)
                    f1 = urllib.request.urlopen(link1)
                    tree1 = ET.parse(f1)
                    doc1 = tree1.getroot()
                    
                    article_authors = []
                    article_pub_type = []
                    for child in doc1:
                        for step_child in child:
                            if step_child.tag == "DocumentSummary":
                                for step_child1 in step_child:
                                    if step_child1.tag == "Authors":
                                        for step_child2 in step_child1:
                                            # Authors
                                            authors = step_child2.find("Name").text
                                            article_authors.append(authors)
                        

                                        
                                    if step_child1.tag == "PubType":
                                        for step_child2 in step_child1:
                                            if step_child2.tag == "flag":
                                                pub_type = step_child2.text
                                                article_pub_type.append(pub_type)
                                        article_pub_type_list.append(article_pub_type)

                        article_authors_list.append(article_authors)
                        volume = step_child.find("Volume").text
                        article_volume_list.append(volume)
                        title = step_child.find("Title").text
                        article_title_list.append(title)
                        pages = step_child.find("Pages").text
                        article_pages_list.append(pages)
                        year = step_child.find("SortPubDate").text
                        article_year_list.append(year)
                        journal = step_child.find("FullJournalName").text
                        article_journal_list.append(journal)                  
                
                
                else:
                    Id_List = []      

            # Count the number of article title matched with PMID on PubMed
            if Id_List != []:
                match_count["counter"] += 1
            else:
                match_count["counter"] += 0
#             PMID_List.append(Id_List)    
            
            

#             pmid_list = " ".join(str(x) for x in Id_List)
#             article_string = title + "," + pmid_list + "\n"
            # Write the result into output file
#                 Result_file.write(article_string)
# PMID_list = [item for sublist in l for item in PMID_list]
df = pd.DataFrame({"PMID": PMID_list, "Title": article_title_list, "Authors": article_authors_list, "Journal": article_journal_list, "Year": article_year_list, "Pages": article_pages_list, "Volume": article_volume_list, "Publication Type": article_pub_type_list})
# df = df[['PMID','Title','Authors','Journal','Year','Pages','Volume','Publication Type']]
# df.to_excel('D:/GRA/Data/PMIDs Mapping Results/PMIDs Mapping Results/Unsuccessful Mapping/Conclusion_changed_processed_unsuccessful.xlsx', sheet_name='sheet1', index=False)

print("\n")
print("Number of articles are:", article_count["counter"])
print("Number of articles matched with PubMed API are:", match_count["counter"])

2492178
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=2492178&version=2.0
10969126
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=10969126&version=2.0


Number of articles are: 41
Number of articles matched with PubMed API are: 2


In [153]:
df.head()

,PMID,Title,Authors,Journal,Year,Pages,Volume,Publication Type
0,[2142109],Placebo-controlled comparison of hormonal and ...,"[Telimaa S, Apter D, Reinilä M, Rönnberg L, Ka...","European journal of obstetrics, gynecology, an...",1990/07/01 00:00,97-105,36,"[Clinical Trial, Journal Article]"
1,[3075526],"[Efficacy of amodiaquine, Fansidar and their c...","[Huang QL, Quyang WC, Zhou JX, Wu Z, Zhang KY,...",Zhongguo ji sheng chong xue yu ji sheng chong ...,1988/01/01 00:00,292-5,6,"[Clinical Trial, Journal Article, Randomized C..."
2,[1872778],Randomized trial of 2 hormonal and 2 prostagla...,"[Fraser IS, McCarron G]",The Australian & New Zealand journal of obstet...,1991/02/01 00:00,66-70,31,"[Clinical Trial, Journal Article, Randomized C..."
3,[3973541],Papanicolaou smear adequacy: effect of differe...,"[Hamblin JE, Brock CD, Litchfield L, Dias J]",The Journal of family practice,1985/03/01 00:00,257-60,20,[Journal Article]
4,[8452061],Papanicolaou smear adequacy: the effect of the...,"[Noel ML, Kazal LA Jr, Glenday MC]",The Journal of the American Board of Family Pr...,1993/03/01 00:00,103-7,6,"[Clinical Trial, Journal Article, Randomized C..."
